In [ ]:
# Imports
from surprise.model_selection import train_test_split
from collections import defaultdict
from surprise.model_selection import GridSearchCV
from surprise import KNNWithMeans
from surprise import get_dataset_dir
from surprise import Dataset
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from scipy.cluster.hierarchy import dendrogram, linkage 
from sklearn.cluster import AgglomerativeClustering  
from sklearn.cluster import KMeans 
import time
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as plt_clrs

%matplotlib inline


# Assignment 2

Welcome to the second assignment! 

You will have to implement clustering, association rules, and recommender systems algorithms, applying these methods to: 
- explore the similarities within groups of people watching movies (clustering analysis)
- discover the relations between movies genre (association rules)
- recommend movies to users (recommender system)

We will use the MovieLens dataset, which contains movie ratings collected from the MovieLens website by the [GroupLens](https://grouplens.org/) research lab.

Source: F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. *ACM Transactions on Interactive Intelligent Systems (TiiS)* 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

Once you are done you have to submit your notebook here: 
[https://moodle.epfl.ch/mod/assign/view.php?id=1247726](https://moodle.epfl.ch/mod/assign/view.php?id=1247726)

If there is need for further clarifications on the questions, after the assignment is released, we will update this file, so make sure you check the git repository for updates.

Good luck!

## Clustering analysis: similarities between people (10 points)

In this section, you will try to form clusters of individuals based on their preferences regarding movie genres. You will use a transformed version of the MovieLens dataset containing, for a selection of users:
- their average rating of all science fiction movies they rated,
- their average rating of all comedy movies they rated.

Better understanding the differences in people's tastes can help improve the design of recommender systems, for instance for the creation of the user neighborhood. Ok, let's start!

- Load the data in a dataframe. The url link is provided below. Display the first 10 observations.

In [ ]:
url_clustering = 'https://raw.githubusercontent.com/michalis0/MGT-502-Data-Science-and-Machine-Learning/main/data/ratings_clustering.csv'
movie_ratings_df = pd.read_csv(url_clustering)

# Displaying 10 first observations
display(movie_ratings_df.head(10))


- Plot a dendogram using 'ward' as linkage method and 'euclidean' as metric. 
- Based on the dendogram, how many clusters do you think is optimal? Briefly justify your answer.

In [ ]:
# Provide the linkage method we want and the chosen distance metric.
X = movie_ratings_df.copy()

method_Z = 'ward'
metric = 'euclidean'
Z = linkage(X, method=method_Z, metric=metric)

# Single linkage
plt.figure(figsize=(16, 6))
dendrogram(Z)  # Plot the dendogram according the linkage
plt.title('Dendrogram (Method: ' + method_Z +
          ', Metric: ' + metric + ')', fontsize=14)
plt.hlines(y=4.5, xmin=0, xmax=2000, colors='k', linestyles='--')
plt.text(x=1000, y=4.75, s='Horizontal line crossing 4 vertical lines', fontsize=12)
plt.xlabel('Index of observations', fontsize=14)
plt.ylabel('Distance', fontsize=14)
plt.show()


If we use the technique seen in the exercises we are looking for the longes vertical lines not crossed by a extended horizontal line. Implementing this technique on this dendrogram we can find four clusters. The four different color-schemes also supports these findings, and makes them easily detectable.

- Implement the Elbow method to determine the optimum number of cluster for K-Means algorithm (use `random_state=17` as parameter of K-Means). 
- Based on the Elbow method, how many clusters do you think is optimal? Briefly justify your answer.

In [ ]:
# Function to calculate elbow of an inertia

def get_elbow_cluster(inertias, staring_cluster=2):
    '''
    Input: Inertias and clusters
    Output: Cluster-elbow
    '''
    data = pd.DataFrame(data=inertias)		# Creating df with inertias
    data['pre'] = data[0].shift(1)			# Setting the previous values with shift 1
    # Setting the succeeding values with shift -1
    data['post'] = data[0].shift(-1)
    data['change'] = (data['post'] - data[0]) + \
        (data['pre'] - data[0]) 	# Calculating the change
    # Finding the datapoint with the highest change compared to the next
    data['elbow'] = data['change'] - data['change'].shift(-1)

    data.sort_values(by=['elbow'], ascending=False, inplace=True)
    data.reset_index(inplace=True)

    elbow_index = data['index'][0]+staring_cluster
    elbow_inertia = data[0][0]

    return elbow_index, elbow_inertia


# Declaring starting variables
n_clusters, start_time, previous_inertia, end_time, inertias = 2, time.time(), 100, 0, []

# Using a while loop that runs for maximum 3 seconds or breaks when reduction becomes smaller than 7,5%
while end_time-start_time < 3:
    km = KMeans(n_clusters=n_clusters, random_state=17, n_init=10).fit(X)
    if previous_inertia/km.inertia_ > 1.075:
        inertias.append(km.inertia_)
        end_time = time.time()
        n_clusters += 1
        previous_inertia = km.inertia_
    else:
        break

# Using helper function to get the 'optimal' number of clusters
elbow_index, elbow_inertia = get_elbow_cluster(inertias)

nbr_clusters = range(2, n_clusters)

# Plot
plt.plot(nbr_clusters, inertias, '-o')
plt.xticks(nbr_clusters)
plt.title('Elbow method for inertia')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.vlines(x=elbow_index, ymin=0, ymax=100, linestyles='--', color='k')
plt.text(x=elbow_index+0.25, y=50,
         s=f'Optimal number of clusters: {elbow_index}', fontsize=10)
plt.text(elbow_index+1, elbow_inertia, f'Elbow Intertia: {round(elbow_inertia, 2)}') 
plt.show()


We can see that after four clusters the reduction in inertia gets lower and lower. I implemented a function called `get_elbow_cluster()` to identify this elbow, where we find the cluster number with the highest difference in change from the previous to the next cluster. I also used a `while` loop to that either runs for a maximum of three seconds or stops when the inertia improvement becomes small (1.075), this way i don't need to specify number of clusters.

- Implement (train) a K-Means algorithm with the number of clusters of your choice. Use `random_state=17` as parameter.

In [ ]:
clusters = 4

# Create K-Means model
kmeans_movie = KMeans(n_clusters=clusters, random_state=17,
                       n_init='auto')  
# Fit  model
kmeans_movie.fit(X)


- Implement (train) a hierarchical algorithm with the same number of clusters as for the K-Means model. Use 'ward' as linkage method and 'euclidean' as metric/affinity 

In [ ]:
# Create Agglomerative Hierarchical model
agglomerative_movie = AgglomerativeClustering(n_clusters=clusters, metric='euclidean', linkage='ward')

# Fit model
agglomerative_movie.fit(X)


- Create a figure consisting of two subplots:
    - a scatterplot of 'avg_scifi_rating' and 'avg_comedy_rating' colored by the clusters predicted with your KMeans model. Add the cluster centers to your plot. Label your clusters with the name of your choice (e.g., 'Comedy aficionado').
    - a scatterplot of 'avg_scifi_rating' and 'avg_comedy_rating' colored by the clusters predicted with your hierarchical algorithm model. Label your clusters with the name of your choice.
- How do your models compare?

In [ ]:
# Creating a df to keep color schemes consistent
df_kmean_agglo = movie_ratings_df.copy()
df_kmean_agglo['kmeans_cluster'] = kmeans_movie.labels_
df_kmean_agglo['agglo_cluster'] = agglomerative_movie.labels_

# Extract columns of choice
x = df_kmean_agglo['avg_scifi_rating']
y = df_kmean_agglo['avg_comedy_rating']
c_kmean = df_kmean_agglo['kmeans_cluster']
c_agglo = df_kmean_agglo['agglo_cluster']

fig, ax = plt.subplots(1, 2, figsize=(14, 4))

# Define the legend labels for the different types of movie wathcers
watcher_labels = ['Poor Comedy Watchers', 'Wannabe Comedians',
                  'Star Wars Fans', 'Undecided']

# Creating scatterplot for the KMean
kmean_scatter = ax[0].scatter(x=x,
                              y=y,
                              c=c_kmean,
                              cmap=plt_clrs.ListedColormap(['lightblue', 'darkblue', 'lightpink', 'lightsalmon']))

# Creating markers for each KMean center
ax[0].scatter(kmeans_movie.cluster_centers_[:, 0],
              kmeans_movie.cluster_centers_[:, 1],
              c='black',
              marker='x')

# Setting title and labels for KMean model
ax[0].set_title(f'KMeans Model with {kmeans_movie.n_clusters} Clusters')
ax[0].set_xlabel('Avg Sci-Fi Rating')
ax[0].set_ylabel('Avg Comendy Rating')
ax[0].legend(handles=kmean_scatter.legend_elements()[0], labels=watcher_labels,
             loc='lower left', bbox_to_anchor=(-0.01, -0.01, 0.1, 0.1))


# Creating scatter for agglomerative model
agglo_scatter = ax[1].scatter(x=x,
                              y=y,
                              c=c_agglo,
                              cmap=plt_clrs.ListedColormap(['lightsalmon', 'darkblue', 'lightpink', 'lightblue']))
ax[1].set_title(
    f'Agglomerative Model with {agglomerative_movie.n_clusters_} Clusters')
ax[1].set_xlabel('Avg Sci-Fi Rating')
ax[1].set_ylabel('Avg Comendy Rating')
ax[1].legend(handles=kmean_scatter.legend_elements()[0], labels=watcher_labels,
             loc='lower left', bbox_to_anchor=(-0.01, -0.01, 0.1, 0.1))

plt.subplots_adjust(wspace=0.4)   # Space between plots
plt.show()


We can see that both models capture the four clusters relatively similar, with the biggest difference being the center data-points. To me it seems that the KMeans model  perform slightly better when only using 4 clusters, as we know that `AgglomerativeClustering` perform better bigger cluster numbers. We know that `AgglomerativeClustering` builds its clusters with a bottom-up approach, where `ward` represents the linkage method. We can see that the agglomerative model has the most uneven cluster sizes, where `Undecided` is the the biggest. This is due to its “rich get richer” behavior. `Ward` is the best strategy for obtaining the most regular cluster sizes, whereas single `linkage` is the worst. Since the data is not new or unseen, the transductive vs inductive difference of these two algorithms does not apply.

## Association Rules: association between movie genres (10 points)

You will now pursue your analysis, but this time trying to dig out information about movies. More precisely, you will search for matches between film genres using association rules. We try to understand, for instance, how likely it is that a film is both drama and action. This information can be interesting for film producers who may either want to produce something similar to the established norm: if most drama films are also action, perhaps the new action-drama film would be equally appreciated, or quite to the contrary try a new combination of genres which is more rare to find.

- Load the data in a dataframe. The url link is provided below. 
- Display the first 10 observations. 
- Print the unique values of genres from the first column. 
- How many unique genres does the first column contain? 
- How many movies does the dataframe contains?

In [ ]:
url_association_rules = 'https://raw.githubusercontent.com/michalis0/MGT-502-Data-Science-and-Machine-Learning/main/data/movies_assoc_rules.csv'

df_association_rules = pd.read_csv(url_association_rules)

display(df_association_rules.head(10))

column = 0
unique_values_in_column = df_association_rules.iloc[:,column].unique()
number_of_genres = len(unique_values_in_column)
number_of_movies = df_association_rules.shape[0]

print(f'The unique values of the first column are: \n{unique_values_in_column}\n')
print(f'Then numbers of unique genres in the column are: {number_of_genres}')
print(f'The dataset contains {number_of_movies} movies')


- Preprocessing: as seen during the lab, convert the dataset using a `Transaction Encoder` from the `mlextend` module so that the dataset is reorganised in columns of unique genres. Rows should contain only True or False boolean values according to whether a film was considered as belonging to a genre column or not. Check that you have the correct dimensions.

In [ ]:
# Convert dataframe to list of list
movie_list = df_association_rules.values.tolist()

# Remove NaNs with list comprehensions
movie_list_cleaned = [[x for x in y if str(x).lower() != 'nan'] for y in movie_list]

# Create instance of Encoder
te = TransactionEncoder()

# Fit encoder and transform our list
movie_list_encoded = te.fit(movie_list_cleaned).transform(movie_list_cleaned)

# Create dataframe with results
movies_encoded = pd.DataFrame(movie_list_encoded, columns=te.columns_)
display(movies_encoded.head())

unique_genres = df_association_rules.describe().loc['unique'].max()      # Number of unique genres
unique_genres_encoded = len(movies_encoded.columns)                      # Number of genres
number_of_movies_encoded = movies_encoded.shape[0]                       # Number of movies

# Doing manual check to check if the data is consistent with each other
if unique_genres_encoded == unique_genres:
    print(f'The number of genres are the same in the database as in the encoded data ({unique_genres})')
    
else:
    print(f'The number of genres are not the same ({unique_genres_encoded}!={unique_genres})')

if number_of_movies == number_of_movies_encoded:
    print(f'We have the same number of movies ({number_of_movies})')
else:
    print(f'We do not have the same number of movies ({number_of_movies}!={number_of_movies_encoded})')

- Frequent itemsets: using the Apriori algorithm to find the frequent itemsets with minimum support of 0.01. There is no condition on the maximum length of an itemset. 
- How many itemsets did the apriori algorithm return above (for min_support=0.01)? 
- What are the 10 itemsets with the largest support (you can directly display a dataframe with the 10 itemsets and their support)?

In [ ]:
# Setting threshold
threshold = 0.01

# Apriori algorithm
freq_items = apriori(movies_encoded, min_support=threshold, use_colnames=True)

# Sorting on the support column and displaying the first 10 itemsets
display(freq_items.sort_values(by=['support'], ascending=False).head(10))
print(f'{freq_items.shape[0]} are above the minimum support trehold of {threshold}')


- Mining for association rules: using the frequent items identified above, find association rules with a minimum confidence of 0.45 and order them by decreasing value of lift.
- Discuss the following statements (true or false with 1-2 lines justification)
    - Animation films are associated with Children.  
    - If a film has the genre Musical, then it is also a Comedy.
    - If War then Drama is the asociation rule with the highest confidence. 

In [ ]:
# Generate rules with 0.45 as minimum confidence
rules = association_rules(freq_items, metric='confidence', min_threshold=0.45)

# Sorting rules by lift 
rules.sort_values(by='lift', ascending=False)


*Discuss statements here* 
- Animation films are associated with Children.  
	- True. The association rule between 'Animation' and 'Children' has a confidence of ~ 0.46, which indicates that there is an association between these two genres. The lift value of ~10.9 also suggests a high likelihood of occurrence of 'Children' genre when 'Animation' genre is present, further supporting the statement, even though the support of ~ 0.08 could be interpreted in the way that it's hard to generalize.
- If a film has the genre Musical, then it is also a Comedy.
	- False. The association rule between 'Musical' and 'Comedy' has a confidence of ~0.48, which is not close to 1, indicating that there is only a moderate association between these two genres. Additionally, the lift value of ~1.53 is also not significantly high, suggesting that the presence of 'Musical' genre does not necessarily guarantee the presence of 'Comedy' genre. 
- If War then Drama is the asociation rule with the highest confidence.
	- True. The association rule between 'War' and 'Drama' has a confidence of ~0.75, which is the highest.



## Recommender systems: item-based recommender system (10 points)

In the walkthrough, we have implemented a user-to-user collaborative filtering algorithm (from scratch and using using Surprise library), i.e., our recommendations were based on the ratings of users with similar tastes. In this assignment, you will implement an **item-to-item** collaborative filtering algorithm, i.e., the recommendations will be based on the set of movies that users like. Do not worry, you won't have to implement the algorithm from scratch and instead can rely on the [Surprise library](http://surpriselib.com/). 

- As in the walkthrough, load the *built-in* `ml-100k` from the Surprise library.

In [ ]:
# Load data
data = Dataset.load_builtin('ml-100k')


- Use GridSearchCV to find the best number of neighbors (k) for a KNNWithMeans **item-based** algorithm, with the following parameters:
    - options for k: `[10, 20, 30, 40, 50]`
    - `'sim_options': {'name': ['pearson'], 'user_based': [???]}` Here you have to replace `???` with the appropriate value...
    - root-mean-square-error (RMSE) as measures,
    - 5 cross-validation folds,
    - other parameters: `refit=True, joblib_verbose=2, n_jobs=-1`
- What is the optimal k for which GridSearchCV returned the best RMSE score? 
- What is the RMSE score for the optimal k?

In [ ]:
param_grid = {'k': [10, 20, 30, 40, 50],
              'sim_options': {'name': ['pearson'], 'user_based': [False]}}

# Using GridSearcgCV to determine the best k
KNN_grid_search = GridSearchCV(KNNWithMeans, param_grid=param_grid,
                               measures=['RMSE'], cv=5,
                               refit=True, joblib_verbose=2, n_jobs=-1)

KNN_grid_search.fit(data)

best_k = KNN_grid_search.best_params['rmse']['k']
best_rmse = KNN_grid_search.best_score['rmse'].round(5)

print(f'\n\nOptimal K: {best_k}')
print(f'Best RMSE: {best_rmse}')


- Using the Surprise library, split your dataset between training and test set. As parameters, use `test_size=0.2, random_state=12`
- Fit a KNNWithMeans algorithm using the best k value retrieved above. As other parameters, use:
    - `min_k=1`
    - `sim_options = {'name': 'pearson','user_based': ???}`
    - `verbose=False`
- Predict ratings on the test set using your algorithm

In [ ]:
sim_options = {
    'name': 'pearson',
    'user_based': False     # user_based = False to capture item-based
}

# Create training and test set
trainset, testset = train_test_split(data, test_size=0.2, random_state=12)

# Instance of KNNWithMeans
knn_means = KNNWithMeans(k=best_k, min_k=1, sim_options=sim_options, verbose=False)

# Fit model on training set
knn_means.fit(trainset)

# Predict ratings on test set
predictions = knn_means.test(testset)


- Use the helper function below to identify the best 10 films for all users
- Find the top 10 predictions for user 169 (you should return the titles of the movies)

In [ ]:
def read_item_names():
    '''Read the u.item file from MovieLens 100-k dataset and return two
    mappings to convert raw ids into movie names and movie names into raw ids.
    '''

    file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
    rid_to_name = {}
    name_to_rid = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = line[1]
            name_to_rid[line[1]] = line[0]

    return rid_to_name, name_to_rid


def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''
    # First map the predictions to each user.
    top_n = defaultdict(list) # This is used to group a sequence of key-value pairs into a dictionary of lists
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n



In [ ]:
# Get top 10 movies for all users
top_n = get_top_n(predictions, n=10)

# Getting mappings from row id to movie name and vice-versa
rid_to_name, name_to_rid = read_item_names()

uid = '169'

# Getting the top 10 movies for uid=169
top_n_movie_ids_for_uid = top_n[uid]

# Getting the first element in each pair (corresponding to each movie id)
recommended_items = [iid for (iid, _) in top_n_movie_ids_for_uid]

# Convert ids into names
item_names = [rid_to_name[rid]
              for rid in recommended_items]

# Print the recommended items for user id 169
print(f'\nTop 10 movies for user with id {uid}:')
print('\n'.join(item_names))


- Plot the precision at rank k and the recall at rank k on the same figure, for k between 0 and 20, and a relevance threshold of 3.75
- Plot the precision-recall curve

*You can, but do not have to, rely on the function(s) used in the lab (i.e., copying the code of the function(s))*

In [ ]:
def precision_recall_at_k(predictions, k, threshold):
    '''
    Compute precision and recall at k metrics for each user in predictions.

    Parameters:
    - predictions: a list of (uid, iid, true_r, est, _) tuples
    - k: the number of top items to consider
    - threshold: the minimum true rating to consider an item as relevant

    Returns:
    - precisions: a dict mapping user ids to their precision@k score
    - recalls: a dict mapping user ids to their recall@k score
    '''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        #user_ratings.sort(key=lambda x: x[0], reverse=True)
        user_ratings.sort(reverse=True)

        # Number of relevant and recommended items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls


def precision_recall_algo(algo, k_range=21, threshold=3.75):
    '''Return precision and recall at k metrics for an algorithm.'''

    # Fit algo on training set
    algo.fit(trainset)

    # Predict on test set
    predictions = algo.test(testset)

    # Compute precision and recall
    precision = []
    recall = []
    for k in range(k_range):
        precisions, recalls = precision_recall_at_k(
            predictions, k=k, threshold=threshold)
        precision.append(
            sum(prec for prec in precisions.values()) / len(precisions))
        recall.append(sum(rec for rec in recalls.values()) / len(recalls))

    return precision, recall


In [ ]:
# Using the previously made model called knn_means and function given in the lab
k_range, rel_threshold = 21, 3.75
precision_KNN, recall_KNN = precision_recall_algo(knn_means, k_range, rel_threshold)

# Plot precision and recall scores
plt.plot(range(k_range), precision_KNN, 'darkblue', label='Precision')
plt.plot(range(k_range), recall_KNN, 'red', label='Recall')
plt.xticks(np.arange(0, k_range, step=1))
plt.xlabel('Rank k')
plt.ylabel('Score')
plt.legend()
plt.title('Precision and Recall for item-to-item based KNN')
plt.show()


In [ ]:
# Plotting Precision-Recall Curve
plt.step(recall_KNN, precision_KNN, color='blue', where='post', label =f'KNN, K: {best_k}, Sim-Options:\n{sim_options})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
plt.title('Precision-Recall Curve')
plt.show()

In [ ]:
%load_ext watermark
%watermark -v -p pandas,numpy,sklearn

Congrats, you are done with the assignment!